<a href="https://colab.research.google.com/github/vhppacheco/ic-ita-finance-ia/blob/main/DFs_Economatica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

#Função de Construção do DataFrame

### A função ***build_df*** tem por objetivo realizar um processo de construção de DataFrames dos **índices Ibovespa e S&P500** através de Screenings e Matrizes constuídas na Plataforma **Economatica**
### Seu único parâmetro é uma *string* ("Ibov" ou "SP500")

In [ ]:
def build_df(indice):

  #Screening
  scr = pd.read_excel(f'Screening_{indice}.xlsx', skiprows = 3)
  codigos= scr['Código']

  #Indicadores
  df0 = pd.read_excel(f'Matrixx_Indicadores_{indice}.xlsx', sheet_name= None, skiprows = 3)

  df = pd.DataFrame()

  for cod in codigos:
    df_temp = df0[cod]
    print(cod)
    df_temp['Código'] = cod
    df = df.append(df_temp)

  #Screening + Indicadores
  df = pd.merge(df,scr, how = 'left', on = 'Código').set_index('Data')
  df.columns = df.columns.str.replace('\\n',' ', regex=True)
  df = df.drop(df.iloc[:,27:38], axis = 1)

  #Cotação Média Trimestral
  cotacao0= pd.read_excel(f'Matrixx_Cotacao_{indice}.xlsx', sheet_name= None, skiprows = 3)

  cotacao_temp0 = pd.DataFrame()

  for cod in codigos:
    cotacao_temp1 = cotacao0[cod]
    print(cod)
    cotacao_temp1['Código'] = cod
    cotacao_temp0 = cotacao_temp0.append(cotacao_temp1)

  cotacao_temp = cotacao_temp0[cotacao_temp0[' Data do Bal\n consolid:sim*'] != '-']
  cotacao_temp = cotacao_temp.replace('-', np.NaN)
  cotacao_temp = cotacao_temp.fillna(method='bfill')

  #DataFrame de Datas + Filtro
  cotacao_temp['Qdate'] = [date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.QuarterEnd() for date in  cotacao_temp.Data]
  datas = cotacao_temp['Qdate'].unique()
  datas = np.sort(datas)

  count_release = cotacao_temp['Código'].groupby(cotacao_temp['Código']).count().sort_values(ascending=False)
  count_release = pd.DataFrame(count_release)
  count_release = count_release[(count_release['Código'] == 24)]

  trimestre = df.index.unique()

  df_datas = list(zip(datas, trimestre))
  df_datas = pd.DataFrame(df_datas, columns=['Data', 'Trimestres'])

  #Cotação Média Trimestral + Datas
  cotacao = cotacao_temp[cotacao_temp.Código.isin(count_release.index)]
  cotacao = cotacao.fillna(method='ffill')
  cotacao = cotacao.set_index('Data')
  cotacao = pd.merge(df_datas, cotacao, how='inner', left_on = 'Data', right_on= ' Data do Bal\n consolid:sim*').set_index('Trimestres')

  #DataFrame Final
  cot = cotacao.reset_index()
  df_final = df.reset_index()
  cot = cot[['Trimestres','Media\ndo fechamento\nem 1 trimestre\nEm moeda orig\najust p/ prov',' Data do Bal\n consolid:sim*','Código']]
  df_final = pd.merge(df_final,cot, how= 'inner', left_on= ['Código','Data'], right_on = ['Código','Trimestres']).set_index('Data')
  df_final.columns = df_final.columns.str.replace('\\n',' ', regex=True)
  df_final = df_final.drop(df_final.columns[[28]], axis = 1)
  df_final = df_final.replace('-', np.NaN).fillna(method='ffill')
  df_final = df_final.sort_values(by=['Código',' Data do Bal  consolid:sim*'], ascending = True)

  return df_final, df_final.to_excel(f'df_{indice}_Economatica.xlsx')

In [ ]:
build_df(indice='Ibov')